In [1]:
import inv_dataset
import browser
import os
import pandas as pd

All Completed, End Task.


In [2]:
inventory_folder = 'data/inventory/'
udpate_folder_path = 'data/updates/'

#convert inventory xls to csv
inventory_files = inv_dataset.Converter()
inventory_files.to_csv(inventory_folder)

#convert updates xls to csv
update_files = inv_dataset.Converter()
update_files.to_csv(udpate_folder_path)

In [3]:
#read inventory csv into dataframe
inventory = inv_dataset.Reader()
inv_df = inventory.get_df(inventory_folder)

In [4]:
'''
back to parent directory if necessery
Use this line: 
os.chdir("../..")

check current directory:
os.getcwd()
'''

os.getcwd()

'c:\\Users\\Waddy Store\\Documents\\Code\\codespaces-flask'

In [10]:
def PreOrderClose():
    process_list = []
    #Start update procedure
    os.chdir(udpate_folder_path)
    #go thru each bulk update form csv
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            df = pd.read_csv(file_path, index_col=None, header=0)
            df = df.drop(index=0)

            df = df.loc[df['Preorder Product'] == "Y"]
            df = df.loc[df['Status'] == "Y"]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] > 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    os.chdir("../..") 
    print(process_list) 
    clicker = browser.Action()
    clicker.close_preorder(process_list)   

In [11]:
PreOrderClose()

[['5f3b50bc09ec050048c12f99', '6971668340378', True], ['6062fa4eec7042002f6a36f0', '7350075028182', True], ['63734a8e89e4d9001cb8b2fc', '4526374213410', False], ['63b65227eb9fed0020588c4e', '6971668340385', True]]
Clicker starts
Login sucessful
Now browsing to SKU: 5f3b50bc09ec050048c12f99
Now browsing to SKU: 6062fa4eec7042002f6a36f0
Now browsing to SKU: 63734a8e89e4d9001cb8b2fc
Now browsing to SKU: 63b65227eb9fed0020588c4e
Go to Variations Tab
Checkbox Status is  None
pass clicking checkbox, already unchecked
Go to Settings Tab
No action, Switch alraedy off
Saved changes, completed


In [14]:
def PreOrderOpen():
    process_list = []
    #Start update procedure
    os.chdir(udpate_folder_path)
    #go thru each bulk update form csv
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            update_df = pd.read_csv(file_path, index_col=None, header=0)
            update_df = update_df.drop(index=0)

            '''
            match certain condition for open pre-order settings
            may add model or brand Keyword (case sensitive here)
            '''
            searchfor = [
                'SN-', 'ARCWAVE', 'HARU', 'LELO', 'Arcwave', 
                '小怪獸', 'BATTLEMAN', 'TS6', 'TITAN', 'ICHIKI','R20',
                'GINRO', 'Relove', 'We-Vibe', '草本精華', 'Womanizer Liberty',
                'Womanizer Starlet', 'Womanizer Premium 2', 'Satisfyer'
                ]

            df = update_df.loc[update_df['Product Name (Traditional Chinese)'].str.contains('|'.join(searchfor), na = False)]

            ''' end conidtion'''

            df = df.loc[df['Preorder Product'] == "N"]
            df = df.loc[df['Status'] == "Y"]

            discon_tag = 'dis'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag, na=False)]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] <= 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    os.chdir("../..") 
    print(process_list) 
    clicker = browser.Action()
    clicker.open_preorder(process_list)   

In [15]:
PreOrderOpen()

[['5f88018535ea09001dbf1c24', '98000165', False], ['6002748c686963001403f155', '98000164', False], ['602df919cd8227001110e7db', '4897086540119', False], ['603a1f2b4d4d860035c53c93', '45451512', False], ['6044d8d0215971001a78e02f', '20649632', False], ['60492a9bca1a330ed27fa253', '21295275', False], ['607d65588aa0ca053c27890b', '60773733', False], ['60801d3c558c4d00327a9b51', '12139790', False], ['611f3a8072786c002924168d', '98000289', True], ['6129cb559e4366002644ce21', '54189922', False], ['6155711e4bff50001d24a49b', '10190541', False], ['61b2304a03f78700293dea8d', '91019531', True], ['61d71d397c3767001db2b267', '72828261', True], ['61e6b3471819fe001d7a4be9', '80056333', True], ['624eeaebb05e95053bf2ddce', '61241800', False], ['625aa88ac36f460018f0fbe1', '54804379', False], ['627b0d8c0b5c1a0021e5f08e', '827160114565', False], ['62b32cd03cc696001b7ecc3d', '66515986', False], ['62d25d23820b69001a3914a5', '54449834', True], ['6329906a35b4d00013e3c2a2', '98000135', False], ['632adcf49f29c